In [2]:
import re
import requests
import pandas as pd
import country_converter as coco

from bs4 import BeautifulSoup
from entsoe import EntsoePandasClient

## Web scraping country codes from Entso website

In [3]:
url = "https://transparency.entsoe.eu/transmission-domain/physicalFlow/show?name=&defaultValue=true&viewType=TABLE&areaType=BORDER_CTY&atch=true&dateTime.dateTime=01.01.2023+00:00|CET|DAY&border.values=CTY|10YAL-KESH-----5!BZN_BZN|10YAL-KESH-----5_BZN_BZN|10YGR-HTSO-----Y&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)&_=1679242106464"

In [4]:
response = requests.get(url)
result = response.content
soup = BeautifulSoup(result, 'html.parser')

In [5]:
countries = soup.find_all("label", {'for': re.compile(r'^border.values')})

In [6]:
country_list = []

for country in countries:
    country_list.append((re.findall("\((.*?)\)", country.get_text())))

country_list

[['AL', 'GR'],
 ['AL', 'XK'],
 ['AL', 'ME'],
 ['AL', 'RS'],
 ['AM', 'GE'],
 ['AT', 'CZ'],
 ['AT', 'DE'],
 ['AT', 'HU'],
 ['AT', 'IT'],
 ['AT', 'SI'],
 ['AT', 'CH'],
 ['AZ', 'GE'],
 ['BY', 'LT'],
 ['BY', 'UA'],
 ['BE', 'FR'],
 ['BE', 'DE'],
 ['BE', 'LU'],
 ['BE', 'NL'],
 ['BE', 'UK'],
 ['BA', 'HR'],
 ['BA', 'ME'],
 ['BA', 'RS'],
 ['BG', 'GR'],
 ['BG', 'MK'],
 ['BG', 'RO'],
 ['BG', 'RS'],
 ['BG', 'TR'],
 ['HR', 'BA'],
 ['HR', 'HU'],
 ['HR', 'RS'],
 ['HR', 'SI'],
 ['CZ', 'AT'],
 ['CZ', 'DE'],
 ['CZ', 'PL'],
 ['CZ', 'SK'],
 ['DK', 'DE'],
 ['DK', 'NL'],
 ['DK', 'NO'],
 ['DK', 'SE'],
 ['EE', 'FI'],
 ['EE', 'LV'],
 ['EE', 'RU'],
 ['FI', 'EE'],
 ['FI', 'NO'],
 ['FI', 'RU'],
 ['FI', 'SE'],
 ['FR', 'BE'],
 ['FR', 'DE'],
 ['FR', 'IT'],
 ['FR', 'ES'],
 ['FR', 'CH'],
 ['FR', 'UK'],
 ['GE', 'AM'],
 ['GE', 'AZ'],
 ['GE', 'RU'],
 ['GE', 'TR'],
 ['DE', 'AT'],
 ['DE', 'BE'],
 ['DE', 'CZ'],
 ['DE', 'DK'],
 ['DE', 'FR'],
 ['DE', 'LU'],
 ['DE', 'NL'],
 ['DE', 'NO'],
 ['DE', 'PL'],
 ['DE', 'SE'],
 ['DE', 'C

# API request

In [7]:
start = pd.Timestamp('20210101', tz='Europe/Brussels')
end   = pd.Timestamp('20220102', tz='Europe/Brussels')

client = EntsoePandasClient(api_key="")

cc = coco.CountryConverter()

### Exports

In [11]:
exports = pd.DataFrame()

for count, value in enumerate(country_list):
    
    try:
        export_temp_df = client.query_crossborder_flows(value[0], value[1], start=start, end=end)
    except :
        print(f"\n{value} not available")
        continue
    
    export_temp_df = export_temp_df.to_frame('Value')
    export_temp_df.insert(0, "Country", value[0])
    export_temp_df.index = export_temp_df.index.tz_localize(None)
    exports = pd.concat([exports, export_temp_df])

    print(value)
    
exports['Country'] = cc.pandas_convert(series=exports['Country'], to='short_name')

['AL', 'GR']
['AL', 'XK']
['AL', 'ME']

['AL', 'RS'] not available

['AM', 'GE'] not available
['AT', 'CZ']
['AT', 'DE']
['AT', 'HU']
['AT', 'IT']
['AT', 'SI']
['AT', 'CH']

['AZ', 'GE'] not available
['BY', 'LT']

['BY', 'UA'] not available
['BE', 'FR']
['BE', 'DE']
['BE', 'LU']
['BE', 'NL']
['BE', 'UK']
['BA', 'HR']
['BA', 'ME']
['BA', 'RS']
['BG', 'GR']
['BG', 'MK']
['BG', 'RO']
['BG', 'RS']
['BG', 'TR']
['HR', 'BA']
['HR', 'HU']
['HR', 'RS']
['HR', 'SI']
['CZ', 'AT']
['CZ', 'DE']
['CZ', 'PL']
['CZ', 'SK']
['DK', 'DE']
['DK', 'NL']
['DK', 'NO']
['DK', 'SE']
['EE', 'FI']
['EE', 'LV']
['EE', 'RU']
['FI', 'EE']
['FI', 'NO']
['FI', 'RU']
['FI', 'SE']
['FR', 'BE']
['FR', 'DE']
['FR', 'IT']
['FR', 'ES']
['FR', 'CH']
['FR', 'UK']

['GE', 'AM'] not available

['GE', 'AZ'] not available

['GE', 'RU'] not available

['GE', 'TR'] not available
['DE', 'AT']
['DE', 'BE']
['DE', 'CZ']
['DE', 'DK']
['DE', 'FR']
['DE', 'LU']
['DE', 'NL']
['DE', 'NO']
['DE', 'PL']
['DE', 'SE']
['DE', 'CH']
['GR', 'A

In [12]:
len(exports)

1887726

In [13]:
exports[0:1032136].to_excel('exports.xlsx')

In [14]:
exports[1032136:-1].to_excel('exports1.xlsx')

###  Imports

In [15]:
imports = pd.DataFrame()

for count, value in enumerate(country_list):
    
    try:
        import_temp_df = client.query_crossborder_flows(value[0], value[1], start=start, end=end)
    except :
        print(f"\n{value} not available")
        continue
    
    import_temp_df = import_temp_df.to_frame('Value')
    import_temp_df.insert(0, "Country", value[1])
    import_temp_df.index = import_temp_df.index.tz_localize(None)
    imports = pd.concat([imports, import_temp_df])

    print(value)
    
imports['Country'] = cc.pandas_convert(series=imports['Country'], to='short_name')

['AL', 'GR']
['AL', 'XK']
['AL', 'ME']

['AL', 'RS'] not available

['AM', 'GE'] not available
['AT', 'CZ']
['AT', 'DE']
['AT', 'HU']
['AT', 'IT']
['AT', 'SI']
['AT', 'CH']

['AZ', 'GE'] not available
['BY', 'LT']

['BY', 'UA'] not available
['BE', 'FR']
['BE', 'DE']
['BE', 'LU']
['BE', 'NL']
['BE', 'UK']
['BA', 'HR']
['BA', 'ME']
['BA', 'RS']
['BG', 'GR']
['BG', 'MK']
['BG', 'RO']
['BG', 'RS']
['BG', 'TR']
['HR', 'BA']
['HR', 'HU']
['HR', 'RS']
['HR', 'SI']
['CZ', 'AT']
['CZ', 'DE']
['CZ', 'PL']
['CZ', 'SK']
['DK', 'DE']
['DK', 'NL']
['DK', 'NO']
['DK', 'SE']
['EE', 'FI']
['EE', 'LV']
['EE', 'RU']
['FI', 'EE']
['FI', 'NO']
['FI', 'RU']
['FI', 'SE']
['FR', 'BE']
['FR', 'DE']
['FR', 'IT']
['FR', 'ES']
['FR', 'CH']
['FR', 'UK']

['GE', 'AM'] not available

['GE', 'AZ'] not available

['GE', 'RU'] not available

['GE', 'TR'] not available
['DE', 'AT']
['DE', 'BE']
['DE', 'CZ']
['DE', 'DK']
['DE', 'FR']
['DE', 'LU']
['DE', 'NL']
['DE', 'NO']
['DE', 'PL']
['DE', 'SE']
['DE', 'CH']
['GR', 'A

In [16]:
len(imports)

1887726

In [21]:
imports[0:1040920].to_excel('imports.xlsx')

In [22]:
imports[1040920:-1].to_excel('imports1.xlsx')